In [ ]:
LGM VIP

In [ ]:
Rashmi S

In [ ]:
Music Recommendation:

In [1]:
import numpy as np
import pandas as pd
from sklearn import model_selection
# import lightgbm as lgb
import os
import sys
import shutil
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
# from catboost import CatBoostClassifier


from lightfm import LightFM
import scipy.sparse as sp


!pip install pyunpack
!pip install patool
from pyunpack.cli import Archive
os.system('apt-get install p7zip')
print(os.getcwd()) #/kaggle/working

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 KB 793.0 kB/s eta 0:00:00
Reading package lists...
Building dependency tree...
Reading state information...
Suggested packages:
  p7zip-full
The following NEW packages will be installed:
  p7zip
0 upgraded, 1 newly installed, 0 to remove and 32 not upgraded.
Need to get 358 kB of archives.
After this operation, 1010 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 p7zip amd64 16.02+dfsg-7build1 [358 kB]


dpkg-preconfigure: unable to re-open stdin: No such file or directory


Fetched 358 kB in 1s (671 kB/s)
Selecting previously unselected package p7zip.
(Reading database ... 105226 files and directories currently installed.)
Preparing to unpack .../p7zip_16.02+dfsg-7build1_amd64.deb ...
Unpacking p7zip (16.02+dfsg-7build1) ...
Setting up p7zip (16.02+dfsg-7build1) ...
Processing triggers for man-db (2.9.1-1) ...
/kaggle/working


In [2]:
import datatable as dt
directory = '/kaggle/working/'
Archive('/kaggle/input/kkbox-music-recommendation-challenge/train.csv.7z').extractall(directory)
Archive('/kaggle/input/kkbox-music-recommendation-challenge/test.csv.7z').extractall(directory)
Archive('/kaggle/input/kkbox-music-recommendation-challenge/songs.csv.7z').extractall(directory)
Archive('/kaggle/input/kkbox-music-recommendation-challenge/members.csv.7z').extractall(directory)
Archive('/kaggle/input/kkbox-music-recommendation-challenge/song_extra_info.csv.7z').extractall(directory)

#sys.exit("Error message")
train = dt.fread('./train.csv').to_pandas()
test = dt.fread('./test.csv').to_pandas()
songs = dt.fread('./songs.csv').to_pandas() #'composer', 'lyricist'
members = dt.fread('./members.csv').to_pandas()

print('Data loading completed!')
print(train.shape, test.shape, songs.shape, members.shape)

Data loading completed!
(7377418, 6) (2556790, 6) (2296320, 7) (34403, 7)


In [3]:
print(train.columns)
print(test.columns)
print(songs.columns)
print(members.columns)

Index(['msno', 'song_id', 'source_system_tab', 'source_screen_name',
       'source_type', 'target'],
      dtype='object')
Index(['id', 'msno', 'song_id', 'source_system_tab', 'source_screen_name',
       'source_type'],
      dtype='object')
Index(['song_id', 'song_length', 'genre_ids', 'artist_name', 'composer',
       'lyricist', 'language'],
      dtype='object')
Index(['msno', 'city', 'bd', 'gender', 'registered_via',
       'registration_init_time', 'expiration_date'],
      dtype='object')


In [4]:
song_cols = ['song_id', 'song_length', 'genre_ids', 'artist_name', 'composer', 'language']
train = train.merge(songs[song_cols], on='song_id', how='left')
test = test.merge(songs[song_cols], on='song_id', how='left')

mem_cols = ['msno', 'city', 'bd', 'gender']
train = train.merge(members[mem_cols], on='msno', how='left')
test = test.merge(members[mem_cols], on='msno', how='left')

for col in [['msno', 'song_id', 'source_system_tab', 'source_screen_name',
             'source_type', 'genre_ids', 'artist_name',
             'composer', 'language', 'city', 'gender']]:
            train[col] = train[col].astype('category')
            test[col] = test[col].astype('category')

In [5]:
for col in train.columns:
    print(train[col].value_counts(), "\n")

    
train = train.drop(['bd', 'msno', 'song_length', 'source_system_tab'], axis = 1)
test = test.drop(['bd', 'msno', 'song_length', 'source_system_tab'], axis = 1)

MXIMDXO0j3UpaT7FvOSGW6Y5zfhlh+xYjTqGoUdMzEE=    5819
o+5RNlSWrzvrphgBNGIo1FLkGxBgyICns6qXj3nS7Pk=    5537
FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=    5511
KGXNZ/H3VxvET/+rGxlrAe7Gpz2eKMXyuSg3xh8Ij1M=    4217
cqjRBV/jWN2ujhc+z/4tz+Mj6xEfflAAt6qBXCqxKvw=    4008
                                                ... 
duhypyeC9me8VBvbMNbCiLfmWorBd8vvQ05O2izI8r4=       1
dtt2VylgQ0l2/4PY0lo0MFdObUJqEecjw/iQSqypY+Q=       1
99S9vfMxOeK+HzFOaklL5sDEsAdUOHxoGdlGwb9R5/8=       1
GUKTK1pT3pWEnlEparWdLZKpCzYaJqBf2dAfoWFDqxQ=       1
YKx6/2NcNyyNLUg0erPb40lYocgqS8R3rZJbqypsmLQ=       1
Name: msno, Length: 30755, dtype: int64 

reXuGcEWDDCnL0K3Th//3DFG4S1ACSpJMzA+CFipo1g=    13973
T86YHdD4C9JSc274b1IlMkLuNdz4BQRB50fWWE7hx9g=    13293
wBTWuHbjdjxnG1lQcbqnK4FddV24rUhuyrYLd9c/hmk=    13079
FynUyq0+drmIARmK1JZ/qcjNZ7DKkqTY6/0O0lTzNUI=    12855
PgRtmmESVNtWjoZHO5a1r21vIz9sVZmcJJpFCbRa1LI=    12004
                                                ...  
RBZxpLNvCiACo2IzK4hv5GC5KvWQgN79ek01ka5xu78=       

In [6]:
train.columns

Index(['song_id', 'source_screen_name', 'source_type', 'target', 'genre_ids',
       'artist_name', 'composer', 'language', 'city', 'gender'],
      dtype='object')

In [7]:
test.columns

Index(['id', 'song_id', 'source_screen_name', 'source_type', 'genre_ids',
       'artist_name', 'composer', 'language', 'city', 'gender'],
      dtype='object')

In [8]:
df_col = [ 'song_id', 'source_screen_name',
       'source_type', 'genre_ids', 'artist_name', 'language', 'city', 'gender']
train = train.drop(['composer'], axis=1)
test = test.drop(['composer'], axis=1)
from sklearn.preprocessing import LabelEncoder

for i in range(len(df_col)):
    train[df_col[i]] = LabelEncoder().fit_transform(train[df_col[i]])
    
for i in range(len(df_col)):
    test[df_col[i]] = LabelEncoder().fit_transform(test[df_col[i]])

In [9]:
from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer()
train = my_imputer.fit_transform(train)

my_imputer = SimpleImputer()
test = my_imputer.fit_transform(test)



In [10]:
train

array([[7.46790e+04, 8.00000e+00, 7.00000e+00, ..., 8.00000e+00,
        0.00000e+00, 0.00000e+00],
       [2.23479e+05, 9.00000e+00, 5.00000e+00, ..., 8.00000e+00,
        1.10000e+01, 1.00000e+00],
       [1.20758e+05, 9.00000e+00, 5.00000e+00, ..., 8.00000e+00,
        1.10000e+01, 1.00000e+00],
       ...,
       [5.15460e+04, 1.60000e+01, 9.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 0.00000e+00],
       [1.02430e+05, 6.00000e+00, 7.00000e+00, ..., 8.00000e+00,
        3.00000e+00, 0.00000e+00],
       [1.23573e+05, 6.00000e+00, 7.00000e+00, ..., 8.00000e+00,
        3.00000e+00, 0.00000e+00]])

In [11]:
test

array([[0.000000e+00, 1.221910e+05, 9.000000e+00, ..., 1.000000e+00,
        0.000000e+00, 0.000000e+00],
       [1.000000e+00, 2.179070e+05, 9.000000e+00, ..., 1.000000e+00,
        0.000000e+00, 0.000000e+00],
       [2.000000e+00, 3.738500e+04, 0.000000e+00, ..., 3.000000e+00,
        0.000000e+00, 0.000000e+00],
       ...,
       [2.556787e+06, 2.194190e+05, 0.000000e+00, ..., 1.000000e+00,
        1.100000e+01, 2.000000e+00],
       [2.556788e+06, 1.213260e+05, 0.000000e+00, ..., 1.000000e+00,
        1.100000e+01, 2.000000e+00],
       [2.556789e+06, 1.543080e+05, 0.000000e+00, ..., 1.000000e+00,
        1.100000e+01, 2.000000e+00]])

In [12]:


train = pd.DataFrame(train, columns = [ 'song_id', 'source_screen_name','source_type', 
                                       'target',  'genre_ids', 'artist_name', 'language', 
                                       'city', 'gender'])
test = pd.DataFrame(test, columns = ['id', 'song_id', 'source_screen_name','source_type', 
                                       'genre_ids', 'artist_name', 'language', 
                                       'city', 'gender'])

In [13]:
test

,id,song_id,source_screen_name,source_type,genre_ids,artist_name,language,city,gender
0,0.0,122191.0,9.0,4.0,301.0,24889.0,1.0,0.0,0.0
1,1.0,217907.0,9.0,4.0,307.0,24742.0,1.0,0.0,0.0
2,2.0,37385.0,0.0,10.0,159.0,21878.0,3.0,0.0,0.0
3,3.0,224360.0,17.0,8.0,307.0,20718.0,8.0,1.0,2.0
4,4.0,85597.0,17.0,8.0,430.0,21761.0,0.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...
2556785,2556785.0,212075.0,12.0,7.0,101.0,19201.0,8.0,11.0,1.0
2556786,2556786.0,78127.0,0.0,7.0,301.0,26482.0,1.0,11.0,2.0
2556787,2556787.0,219419.0,0.0,7.0,301.0,23457.0,1.0,11.0,2.0
2556788,2556788.0,121326.0,0.0,7.0,307.0,26763.0,1.0,11.0,2.0


In [14]:
train = train.astype(int)
test = test.astype(int)

In [15]:
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split

X = train
X = X.drop(['target'], axis = 1)
y = train[['target']]

print(X.head())
print(y.head())

   song_id  source_screen_name  source_type  genre_ids  artist_name  language  \
0    74679                   8            7        286         3277         8   
1   223479                   9            5         91        31960         8   
2   120758                   9            5         91        21372         8   
3    23707                   9            5          7        27439         0   
4    33308                   8            7          3         4472         8   

   city  gender  
0     0       0  
1    11       1  
2    11       1  
3    11       1  
4     0       0  
   target
0       1
1       1
2       1
3       1
4       1


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30)
clf = RandomForestClassifier(n_estimators = 16)
clf.fit(X_train, y_train.values.ravel())
y_pred = clf.predict(X_test)
from sklearn import metrics 
print()

# using metrics module for accuracy calculation
print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(y_test, y_pred))


ACCURACY OF THE MODEL:  0.6204282798051352


In [17]:
pred = clf.predict(test.drop(['id'], axis = 1))


subm = pd.DataFrame()
subm['id'] = test['id']
subm['target'] = pred

subm

,id,target
0,0,1
1,1,1
2,2,0
3,3,0
4,4,0
...,...,...
2556785,2556785,0
2556786,2556786,1
2556787,2556787,0
2556788,2556788,0


In [18]:
subm.to_csv('submission2.csv.gz', compression = 'gzip', index=False, float_format = '%.5f')
print('Done!')

Done!
